In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px
from energyPlus.weather.weather import *

In [ ]:
data, meta, climate_zones = getWeatherData()
#change year to 2000 for all datetimes in index
data.index = data.index.map(lambda x: x.replace(year=2000))
data.keys()

In [ ]:
fig, ax = plt.subplots()
for zone in climate_zones:
    zoneData = data[data["ClimateZone"]==zone]
    ax.plot(zoneData.index, zoneData["temp_air"], label=zone)
ax.legend()

In [ ]:
poa = getRadData(data, meta, tilts=[0], azimuths=[0])

# Plot every column in the poa DataFrame
plt.figure(figsize=(10, 6))
for column in poa.columns:
    plt.plot(poa.index, poa[column], label=column)

plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Time')
plt.ylabel('Irradiance on Horizontal Surface')
plt.title('POA Irradiance for All Columns')
plt.tight_layout()
plt.show()


In [ ]:
poa = getRadData(data, meta, tilts=[90], azimuths=range(0, 360, 5))

# Plot every column in the poa DataFrame
plt.figure(figsize=(10, 6))
for column in poa.columns:
    plt.plot(poa.index, poa[column], label=column)

plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Time')
plt.ylabel('Irradiance on Average Vertical Surface')
plt.title('POA Irradiance for All Columns')
plt.tight_layout()
plt.show()

In [ ]:
chosenWeatherProperties = {}
chosenData = []
toPlot = True
N = 10000
n = 0
for n in range(N):
    weatherProperties, dataSampled = sampleVentWeather(data, climate_zones, runDays=7, dt=3600, plot=toPlot, coolingThreshold=24, coolingDegBase=21, keep="CDDs")
    for key, value in weatherProperties.items():
        if n == 0:
            chosenWeatherProperties[key] = [value]
        else:
            chosenWeatherProperties[key].append(value)
    chosenData.append(dataSampled)
    toPlot = False


In [ ]:
weather = pd.DataFrame(chosenWeatherProperties)
weather.columns = ["Climate Zone", "Month", "CDDs", "NCDDs", "Average Wind Speed [m/s]", "Days Checked"]
weather["Total Days"] = weather["Days Checked"].sum()
print(weather["Total Days"].values[0])

weights = None
plotWeatherHists(weather, weights, ["Month", "Average Wind Speed [m/s]"])
plotWeatherHists(weather, weights, ["CDDs", "NCDDs"])
plotWeatherHists(weather, weights, ["Climate Zone", "Days Checked"])

In [ ]:
plotWeatherHists(weather, 'CDDs', ["Month", "Average Wind Speed [m/s]"], yearScaling = 52.1429 / weather.shape[0])
plotWeatherHists(weather, 'NCDDs', ["Month", "Average Wind Speed [m/s]"], yearScaling = 52.1429 / weather.shape[0])

In [ ]:
plotCZMap(weather, weights="NCDDs", vmax=None, colormap = "PuBu")
plotCZMap(weather, weights="CDDs", vmax=1800, colormap = "PuBu")